Entrenamiento de YOLO para detección de motocicletas fáciles de manejar.

instalación de librerías necesarias. Se tiene diferentes recursos en GCP por eso se instala el cliente de Storage para descargar fácilmente esa información

In [15]:
%pip install google-cloud-storage
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.
  Using cached ultralytics-8.3.194-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.17-py3-none-any.whl.metadata (14 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached n

Descardar el Dataset del Bucket en GCP usando la cuenta de servicio en llave.json

In [8]:
from google.cloud import storage

# Set the path to your service account key file
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/kaggle/input/llave2/vision-scooter-4932d08c8db1.json'

# Initialize a storage client
storage_client = storage.Client()

# Define the bucket name and the source prefix (folder) in the bucket
bucket_name = 'vision-scooter-raw'
file_name = 'yolo_dataset3.zip'

# Define the destination directory on your local machine
destination_directory = '/kaggle/working/dataset.zip'


# Get the bucket
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the file to the local machine
blob.download_to_filename(destination_directory)
# List all blobs (files) in the specified prefix
# blobs = bucket.list_blobs(prefix=source_prefix)

# # Download each blob to the destination directory
# for blob in blobs:
#     # Construct the destination file path
#     # We remove the source_prefix from the blob name to maintain the folder structure
#     destination_file_name = blob.name[len(source_prefix):]
#     destination_file_path = os.path.join(destination_directory, destination_file_name)

#     # Create the necessary subdirectories if they don't exist
#     os.makedirs(os.path.dirname(destination_file_path), exist_ok=True)

#     # Download the blob
#     if not blob.name.endswith('/'): # Avoid trying to download folders themselves
#         print(f"Downloading {blob.name} to {destination_file_path}")
#         blob.download_to_filename(destination_file_path)

print("Download complete.")

Download complete.


Descomprimir el datset para poderlo usar

In [9]:
!unzip dataset.zip -d /kaggle/working/


Archive:  dataset.zip
   creating: /kaggle/working/dataset3/
   creating: /kaggle/working/dataset3/labels/
   creating: /kaggle/working/dataset3/labels/test/
  inflating: /kaggle/working/dataset3/labels/test/bc6fa875-0acb-4842-979d-80b3970b7235.txt  
  inflating: /kaggle/working/dataset3/labels/test/345e0ce9-e942-417b-b4ed-5274aabb66b4.txt  
  inflating: /kaggle/working/dataset3/labels/test/ea4c0194-4c91-4efd-830b-afb6f6402e75.txt  
  inflating: /kaggle/working/dataset3/labels/test/11036cac-2ffc-422d-86ae-7f4c0d154fe2.txt  
  inflating: /kaggle/working/dataset3/labels/test/9418f0e5-cf87-4950-acc9-b85e4e70410e.txt  
  inflating: /kaggle/working/dataset3/labels/test/03d1a650-2141-4954-a685-722926dc6561.txt  
  inflating: /kaggle/working/dataset3/labels/test/d4d3b6b1-cbb0-47f3-b795-552e06a4438b.txt  
  inflating: /kaggle/working/dataset3/labels/test/bc4dbb82-d8c9-4039-b587-ef620132a6c2.txt  
  inflating: /kaggle/working/dataset3/labels/test/c2f90da3-db08-4a9b-8c73-54eb4bde66cf.txt  
  inf

In [11]:
!mv /kaggle/working/dataset3 /kaggle/working/dataset


Creación de dataset.yaml para indicarle a YOLO donde y como se compone el conjunto de datos para el entrenamiento

In [12]:
yaml_content = """path: /kaggle/working/dataset # dataset root dir
train: images/train
val: images/val
test: images/test

# Classes
names:
  0: Faciles
  1: Otras
"""

with open('/kaggle/working/dataset/dataset.yaml', 'w') as f:
    f.write(yaml_content)

print("Created /kaggle/working/dataset/dataset.yaml")

Created /kaggle/working/dataset/dataset.yaml


Yolo fue entrenado de manera de poder salvar los chekpoints en GCP, por eso se corren pocas épocas y luego se retoma el experimento.

In [9]:
##### from ultralytics import YOLO
from datetime import datetime
import shutil
import os
from google.cloud import storage
from ultralytics import YOLO

modelo = YOLO("yolo11x-seg.pt")
PROJECT_NAME = 'experimento1_yolo11'
NAME_PREFIX = "experimento2_"

total_epochs= 100
run_epochs = 20
actual_epoch = 0

storage_client = storage.Client()
bucket_name = 'vision-scooter-raw'


last_name= ""
while actual_epoch < total_epochs:
  print(f"Epoch {actual_epoch}")
    
  name = f"{NAME_PREFIX}{datetime.now().strftime('%Y%m%d_%H%M%S')}"

  if actual_epoch !=0:
      
    print(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")
    modelo = YOLO(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/best.pt")

  epochs =  min(run_epochs, total_epochs - actual_epoch)
  print(epochs,PROJECT_NAME, name )

  modelo.train(data='/kaggle/working/dataset/dataset.yaml', epochs=epochs, imgsz=640, project=PROJECT_NAME, name=name, device=[0, 1], patience = 5)  # train the model

  source_dir = f"/kaggle/working/{PROJECT_NAME}/{name}"
  destination_zip = f"/kaggle/working//{PROJECT_NAME}/{name}.zip"

  # Create the zip file
  shutil.make_archive(destination_zip.replace('.zip', ''), 'zip', source_dir)

  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(f"{name}.zip")
  blob.upload_from_filename(destination_zip)

    

  print(f"Created zip file: {destination_zip}")

  last_name = name

  actual_epoch += epochs



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Epoch 0
20 experimento1_yolo11 experimento2_20250901_032450
Ultralytics 8.3.190 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.

libpng warning: iCCP: known incorrect sRGB profile


       1/20      9.43G     0.5079      1.121      1.228      1.062         25        640: 19% ━━────────── 61/324 1.1it/s 55.8s<3:53

libpng warning: iCCP: known incorrect sRGB profile


       1/20      9.43G     0.5487      1.152      1.173      1.082         28        640: 25% ━━╸───────── 80/324 1.1it/s 1:13<3:430

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 76% ━━━━━━━━━─── 31/41 2.0it/s 14.3s<5.1s

libpng warning: iCCP: known incorrect sRGB profile


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 41/41 2.0it/s 20.1s0.6s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        647        692      0.954      0.957      0.971      0.924      0.954      0.957       0.97      0.881
               Faciles        311        343      0.942      0.944      0.966       0.93      0.942      0.944      0.965      0.887
                 Otras        336        349      0.966       0.97      0.975      0.918      0.966       0.97      0.975      0.876
Speed: 0.2ms preprocess, 26.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to experimento1_yolo11/experimento2_20250901_075550
Created zip file: /kaggle/working//experimento1_yolo11/experimento2_20250901_075550.zip


En el experimento 3 se decide poner una semilla fija que ayuda a retomar el entrenamiento. Se reduce el Batch Size a la mitad para liberar un poco la carga de las GPUs

In [ ]:
##### from ultralytics import YOLO
from datetime import datetime
import shutil
import os
from google.cloud import storage
from ultralytics import YOLO

modelo = YOLO("yolo11x-seg.pt")
PROJECT_NAME = 'experimento3_yolo11'
NAME_PREFIX = "experimento3_"

total_epochs= 100
run_epochs = 20
actual_epoch = 0

storage_client = storage.Client()
bucket_name = 'vision-scooter-raw'


last_name= ""
while actual_epoch < total_epochs:
  print(f"Epoch {actual_epoch}")
    
  name = f"{NAME_PREFIX}{datetime.now().strftime('%Y%m%d_%H%M%S')}"

  if actual_epoch !=0:
      
    print(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")
    modelo = YOLO(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/best.pt")

  epochs =  min(run_epochs, total_epochs - actual_epoch)
  print(epochs,PROJECT_NAME, name )

  modelo.train(data='/kaggle/working/dataset/dataset.yaml', epochs=epochs, imgsz=640, project=PROJECT_NAME, name=name, device=[0, 1], patience = 5, seed= 43, batch = 8)  # train the model

  source_dir = f"/kaggle/working/{PROJECT_NAME}/{name}"
  destination_zip = f"/kaggle/working//{PROJECT_NAME}/{name}.zip"

  # Create the zip file
  shutil.make_archive(destination_zip.replace('.zip', ''), 'zip', source_dir)

  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(f"{name}.zip")
  blob.upload_from_filename(destination_zip)

    

  print(f"Created zip file: {destination_zip}")

  last_name = name

  actual_epoch += epochs


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Epoch 0
20 experimento3_yolo11 experimento3_20250902_004620
Ultralytics 8.3.191 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0

libpng warning: iCCP: known incorrect sRGB profile


       1/20      5.52G     0.5778      1.185      1.276      1.125         16        640: 20% ━━────────── 127/648 2.0it/s 1:05<4:191

libpng warning: iCCP: known incorrect sRGB profile


       1/20      5.54G     0.6364      1.254      1.243      1.151         11        640: 26% ━━━───────── 166/648 2.1it/s 1:24<3:51

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 83% ━━━━━━━━━╸── 67/81 3.8it/s 15.2s<3.7s

libpng warning: iCCP: known incorrect sRGB profile


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 81/81 4.2it/s 19.4s0.3s
                   all        647        692      0.954      0.921      0.967      0.895      0.955      0.922      0.966      0.836

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
       6/20      5.73G     0.5431      1.134     0.5543       1.06          8        640: 14% ━╸────────── 92/648 2.1it/s 43.2s<4:21

libpng warning: iCCP: known incorrect sRGB profile


       6/20      5.73G     0.5437      1.111     0.5565      1.063         17        640: 18% ━━────────── 115/648 2.1it/s 53.9s<4:14

Experimento 4: Hace falta entrenamiento, la estrategia de retomar el mejor modelo se chekpoint anterior no sirve. Se agra Dropout y se disminuye el learning rate para ver si el modelo aumenta su precisión 

In [ ]:
##### from ultralytics import YOLO
from datetime import datetime
import shutil
import os
from google.cloud import storage
from ultralytics import YOLO

modelo = YOLO("yolo11x-seg.pt")
PROJECT_NAME = 'experimento3_yolo11'
NAME_PREFIX = "experimento3_"

total_epochs= 100
run_epochs = 50
actual_epoch = 0

storage_client = storage.Client()
bucket_name = 'vision-scooter-raw'


last_name= ""
while actual_epoch < total_epochs:
  print(f"Epoch {actual_epoch}")
    
  name = f"{NAME_PREFIX}{datetime.now().strftime('%Y%m%d_%H%M%S')}"

  if actual_epoch !=0:
      
    print(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")
    modelo = YOLO(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")

  epochs =  min(run_epochs, total_epochs - actual_epoch)
  print(epochs,PROJECT_NAME, name )

  modelo.train(data='/kaggle/working/dataset/dataset.yaml', epochs=epochs, imgsz=640, project=PROJECT_NAME, name=name, device=[0, 1], patience = 5, 
               seed= 43, batch = 4, lr0=0.005, dropout = 0.2 )  # train the model

  source_dir = f"/kaggle/working/{PROJECT_NAME}/{name}"
  destination_zip = f"/kaggle/working//{PROJECT_NAME}/{name}.zip"

  # Create the zip file
  shutil.make_archive(destination_zip.replace('.zip', ''), 'zip', source_dir)

  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(f"{name}.zip")
  blob.upload_from_filename(destination_zip)

    

  print(f"Created zip file: {destination_zip}")

  last_name = name

  actual_epoch += epochs

Epoch 0
50 experimento3_yolo11 experimento3_20250902_235826
Ultralytics 8.3.191 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, nam

libpng warning: iCCP: known incorrect sRGB profile


       1/50      3.59G     0.6673       1.27      1.339      1.194          5        640: 20% ━━────────── 260/1296 3.2it/s 1:30<5:267

libpng warning: iCCP: known incorrect sRGB profile


       1/50      3.59G     0.7327      1.349      1.356      1.225          5        640: 26% ━━━───────── 338/1296 3.2it/s 1:55<4:58

...

       3/50      4.96G     0.4849     0.9307     0.5241      1.039          4        640: 50% ━━━━━━────── 648/1296 3.3it/s 3:17<3:16

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB


       3/50      4.96G     0.4808     0.9161     0.5163      1.036          4        640: 57% ━━━━━━╸───── 735/1296 3.4it/s 3:43<2:46

libpng warning: iCCP: known incorrect sRGB profile


       3/50      4.96G     0.4878     0.9208     0.5178       1.04          5        640: 65% ━━━━━━━╸──── 842/1296 3.2it/s 4:15<2:22

Experimento 5. Se usa un nuevo dataset corrigiendo uno errores de etiquetado, también se disminuye el learnig rate y se aumenta el dropout 

In [16]:
##### from ultralytics import YOLO
from datetime import datetime
import shutil
import os
from google.cloud import storage
from ultralytics import YOLO

modelo = YOLO("yolo11x-seg.pt")
PROJECT_NAME = 'experimento5_yolo11'
NAME_PREFIX = "experimento5_"

total_epochs= 200
run_epochs = 100
actual_epoch = 0

storage_client = storage.Client()
bucket_name = 'vision-scooter-raw'


last_name= ""
while actual_epoch < total_epochs:
  print(f"Epoch {actual_epoch}")
    
  name = f"{NAME_PREFIX}{datetime.now().strftime('%Y%m%d_%H%M%S')}"

  if actual_epoch !=0:
      
    print(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")
    modelo = YOLO(f"/kaggle/working/{PROJECT_NAME}/{last_name}/weights/last.pt")

  epochs =  min(run_epochs, total_epochs - actual_epoch)
  print(epochs,PROJECT_NAME, name )

  modelo.train(data='/kaggle/working/dataset/dataset.yaml', epochs=epochs, imgsz=640, project=PROJECT_NAME, name=name, device=[0, 1], patience = 10, 
               seed= 43, batch = 8, lr0=0.0025, dropout = 0.25 )  # train the model

  source_dir = f"/kaggle/working/{PROJECT_NAME}/{name}"
  destination_zip = f"/kaggle/working//{PROJECT_NAME}/{name}.zip"

  # Create the zip file
  shutil.make_archive(destination_zip.replace('.zip', ''), 'zip', source_dir)

  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(f"{name}.zip")
  blob.upload_from_filename(destination_zip)

    

  print(f"Created zip file: {destination_zip}")

  last_name = name

  actual_epoch += epochs

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Epoch 0
100 experimento5_yolo11 experimento5_20250906_021333
Ultralytics 8.3.194 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.25, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=

libpng warning: iCCP: known incorrect sRGB profile


      1/100      5.52G     0.5778      1.185      1.276      1.125         16        640: 20% ━━────────── 127/648 2.0it/s 1:06<4:219

libpng warning: iCCP: known incorrect sRGB profile


      1/100      5.54G     0.6364      1.254      1.243      1.151         11        640: 26% ━━━───────── 166/648 2.0it/s 1:25<3:55

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 83% ━━━━━━━━━╸── 67/81 3.5it/s 16.0s<4.0s

libpng warning: iCCP: known incorrect sRGB profile


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 81/81 3.9it/s 20.7s0.3s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        647        692      0.954      0.942      0.974      0.936      0.954      0.942      0.975      0.902
               Faciles        311        343      0.966      0.913       0.97      0.938      0.966      0.913       0.97      0.908
                 Otras        336        349      0.942       0.97      0.979      0.934      0.942       0.97      0.979      0.897
Speed: 0.2ms preprocess, 27.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /kaggle/working/experimento5_yolo11/experimento5_20250906_104150
Created zip file: /kaggle/working//experimento5_yolo11/experimento5_20250906_104150.zip
